# AES-GCM Nonce Reuse Attack

Two ciphertexts encrypted with AES-GCM using the same iv is vulnerable to a nonce reuse attack; if one of the plaintexts is known, the other plaintext can be easily solved for. The xor of the two ciphertexts is equivalent to the xor of the plaintexts. So xor-ing this again to the known plaintext reveals the other plaintext.

Resources:

https://github.com/ashutosh1206/Crypton/blob/master/Authenticated-Encryption/AES-GCM/AES-GCM-implementation.py

https://www.elttam.com/blog/key-recovery-attacks-on-gcm/#content

In [17]:
base64abc = [
    "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M",
    "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
    "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "/"
]

base64codes = [
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 62, 255, 255, 255, 63,
    52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 255, 255, 255, 0, 255, 255,
    255, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
    15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 255, 255, 255, 255, 255,
    255, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
    41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51
]

In [18]:
def base64ToBytes(string):
    if len(string) % 4 != 0:
        print("Unable to parse base64 string.")
        return
    index = string.index("=")
    if index != -1 and index < len(string) - 2:
        print("Unable to parse base64 string.")
        return
    missingOctets = 0
    if string[-2:] == "==":
        missingOctets = 2
    elif string[-1] == "=":
        missingOctets = 1
    else:
        missingOctets = 0
    n = len(string)
    # result = [0 for x in range(3 * (n / 4))]
    result = [0 for x in range(int(3 * (n / 4)))]
    buffer = None
    i = 0
    j = 0
    while i < n:
        buffer = (
            base64codes[ord(string[i])] << 18
            | base64codes[ord(string[i + 1])] << 12
            | base64codes[ord(string[i + 2])] << 6
            | base64codes[ord(string[i + 3])]
        )
        result[j] = buffer >> 16
        result[j + 1] = (buffer >> 8) & 0xFF
        result[j + 2] = buffer & 0xFF
        i += 4
        j += 3
    return result[: len(result) - missingOctets]


def bytesToBase64(bytes):
    result = ""
    i = 2
    l = len(bytes)
    for i in range(2, l, 3):
        result += base64abc[bytes[i - 2] >> 2]
        result += base64abc[((bytes[i - 2] & 0x03) << 4) | (bytes[i - 1] >> 4)]
        result += base64abc[((bytes[i - 1] & 0x0F) << 2) | (bytes[i] >> 6)]
        result += base64abc[bytes[i] & 0x3F]
    i += 3
    if i == l + 1:
        result += base64abc[bytes[i - 2] >> 2]
        result += base64abc[(bytes[i - 2] & 0x03) << 4]
        result += "=="
    if i == l:
        result += base64abc[bytes[i - 2] >> 2]
        result += base64abc[((bytes[i - 2] & 0x03) << 4) | (bytes[i - 1] >> 4)]
        result += base64abc[(bytes[i - 1] & 0x0F) << 2]
        result += "="
    return result

In [19]:
def xor(s1, s2):
    """
    XOR two strings s1, s2
    Strings s1 and s2 need not be of equal length
    Whatever be the length of two strings, the longer string will be sliced
    to a string of length equal to that of the shorter string
    """
    if len(s1) == len(s2):
        return [i ^ j for i, j in zip(s1, s2)]
    elif len(s1) > len(s2):
        return [i ^ j for i, j in zip(s1[: len(s2)], s2)]
    elif len(s1) < len(s2):
        return [i ^ j for i, j in zip(s1, s2[: len(s1)])]

In [ ]:
# %pip install python-dotenv

import os
from dotenv import load_dotenv
load_dotenv()

b64_cipher1 = os.getenv("CIPHER1")
b64_cipher2 = os.getenv("CIPHER2")

cipher1 = base64ToBytes(b64_cipher1)
cipher1_iv = cipher1[0:12]
cipher1_data = cipher1[12:]

cipher2 = base64ToBytes(b64_cipher2)
cipher2_iv = cipher2[0:12]
cipher2_data = cipher2[12:]

xor_data = xor(cipher1_data, cipher2_data)

plain1 = os.getenv("PLAIN1")
bytes_plain1 = bytearray()
bytes_plain1.extend(map(ord, plain1))

bytes_plain2 = xor(bytes_plain1, xor_data)
plain2 = bytes(bytes_plain2)

plain2